# 02.02 - Funciones Avanzadas en Python

**Autor:** Miguel Angel Vazquez Varela  
**Nivel:** Fundamentos  
**Tiempo estimado:** 30 min

---

## ¿Qué aprenderás?

- Argumentos variables con `*args` y `**kwargs`
- Funciones de orden superior: `map()`, `filter()`
- Closures: funciones que recuerdan su entorno
- Decoradores: modificar comportamiento sin tocar el código original

---

## 1. `*args` — Argumentos posicionales variables

`*args` permite que una función acepte **cualquier número de argumentos posicionales**.  
Dentro de la función, `args` es una **tupla**.

In [1]:
def total_distance(*args):
    """Suma la distancia de cualquier numero de viajes."""
    return sum(args)

print(total_distance(5, 3))           # 2 viajes
print(total_distance(5, 3, 12, 8))    # 4 viajes
print(total_distance(2))              # 1 viaje

8
28
2


In [2]:
def trip_report(station, *durations):
    """Genera un reporte de viajes desde una estacion."""
    print(f"Estacion: {station}")
    print(f"Viajes registrados: {len(durations)}")
    if durations:
        print(f"Duracion media: {sum(durations) / len(durations):.1f} min")

trip_report("Sol", 12, 25, 8, 30, 15)

Estacion: Sol
Viajes registrados: 5
Duracion media: 18.0 min


---

## 2. `**kwargs` — Argumentos nominales variables

`**kwargs` permite pasar **cualquier número de argumentos con nombre**.  
Dentro de la función, `kwargs` es un **diccionario**.

In [3]:
def create_trip_record(**kwargs):
    """Crea un registro de viaje con los campos que se pasen."""
    record = {}
    for key, value in kwargs.items():
        record[key] = value
    return record

trip = create_trip_record(origin="Sol", destination="Atocha", duration=18, bike_id=42)
print(trip)

{'origin': 'Sol', 'destination': 'Atocha', 'duration': 18, 'bike_id': 42}


In [4]:
# kwargs es util para funciones de configuracion
def configure_filter(df, **filters):
    """Filtra un DataFrame por columnas y valores dinamicamente."""
    result = df.copy()
    for column, value in filters.items():
        result = result[result[column] == value]
    return result

import pandas as pd

trips = pd.DataFrame({
    'station': ['Sol', 'Atocha', 'Sol', 'Cibeles'],
    'day': ['Mon', 'Mon', 'Tue', 'Mon'],
    'duration': [12, 25, 8, 30]
})

# Filtrar por estacion Y dia
result = configure_filter(trips, station='Sol', day='Mon')
print(result)

  station  day  duration
0     Sol  Mon        12


---

## 3. Combinando `*args` y `**kwargs`

El orden correcto en la firma es: `def func(pos, *args, **kwargs)`

In [5]:
def log_event(event_type, *items, **metadata):
    """
    Registra un evento con items y metadatos opcionales.
    
    Parameters
    ----------
    event_type : str
        Tipo de evento
    *items : str
        Items afectados por el evento
    **metadata : any
        Informacion extra sobre el evento
    """
    print(f"[{event_type.upper()}]")
    print(f"  Items: {items}")
    for k, v in metadata.items():
        print(f"  {k}: {v}")

log_event("rental", "bike_001", "bike_042", station="Sol", user_id=99)

[RENTAL]
  Items: ('bike_001', 'bike_042')
  station: Sol
  user_id: 99


### Desempaquetar listas y dicts al llamar funciones

In [6]:
def calculate_speed(distance_km, duration_min):
    return distance_km / (duration_min / 60)

# Desempaquetar una lista con *
params_list = [5, 30]
print(f"Velocidad: {calculate_speed(*params_list):.2f} km/h")

# Desempaquetar un dict con **
params_dict = {'distance_km': 8, 'duration_min': 20}
print(f"Velocidad: {calculate_speed(**params_dict):.2f} km/h")

Velocidad: 10.00 km/h
Velocidad: 24.00 km/h


---

## 4. Funciones de orden superior: `map()` y `filter()`

Las funciones de orden superior reciben o retornan otras funciones.

In [7]:
durations = [5, 12, 35, 8, 22, 45, 3, 18]

# map(): aplica una funcion a cada elemento
def to_hours(minutes):
    return round(minutes / 60, 2)

in_hours = list(map(to_hours, durations))
print("En horas:", in_hours)

En horas: [0.08, 0.2, 0.58, 0.13, 0.37, 0.75, 0.05, 0.3]


In [8]:
# filter(): conserva solo los elementos que pasan un test
long_trips = list(filter(lambda d: d > 20, durations))
print("Viajes largos (>20 min):", long_trips)

# Equivalente con list comprehension (mas pythonico)
long_trips_lc = [d for d in durations if d > 20]
print("Equivalente con list comp:  ", long_trips_lc)

Viajes largos (>20 min): [35, 22, 45]
Equivalente con list comp:   [35, 22, 45]


In [9]:
# Aplicar a un DataFrame con .apply()
import pandas as pd

df = pd.DataFrame({'duration_min': durations})

df['category'] = df['duration_min'].apply(
    lambda d: 'largo' if d > 20 else ('medio' if d > 10 else 'corto')
)
df['duration_h'] = df['duration_min'].map(to_hours)

print(df)

   duration_min category  duration_h
0             5    corto        0.08
1            12    medio        0.20
2            35    largo        0.58
3             8    corto        0.13
4            22    largo        0.37
5            45    largo        0.75
6             3    corto        0.05
7            18    medio        0.30


---

## 5. Closures

Un **closure** es una función que recuerda las variables del ámbito donde fue creada,  
incluso después de que ese ámbito haya terminado.

In [10]:
def make_classifier(threshold):
    """Crea un clasificador de viajes con un umbral especifico."""
    def classify(duration):
        return "largo" if duration > threshold else "corto"
    return classify

# Crear clasificadores con distintos criterios
classify_strict = make_classifier(threshold=15)
classify_relaxed = make_classifier(threshold=30)

for d in [10, 20, 35]:
    print(f"{d} min — estricto: {classify_strict(d)}, relajado: {classify_relaxed(d)}")

10 min — estricto: corto, relajado: corto
20 min — estricto: largo, relajado: corto
35 min — estricto: largo, relajado: largo


In [11]:
# Uso practico: generador de descuentos
def make_discount(pct):
    """Crea una funcion que aplica un descuento fijo."""
    factor = 1 - pct / 100
    def apply(price):
        return round(price * factor, 2)
    return apply

discount_10 = make_discount(10)
discount_25 = make_discount(25)

price = 2.50  # precio base de alquiler
print(f"Precio base: {price}€")
print(f"Con 10% dto: {discount_10(price)}€")
print(f"Con 25% dto: {discount_25(price)}€")

Precio base: 2.5€
Con 10% dto: 2.25€
Con 25% dto: 1.88€


---

## 6. Decoradores

Un **decorador** es un closure que envuelve una función para añadir comportamiento  
sin modificar su código original. Se aplica con la sintaxis `@nombre_decorador`.

In [12]:
import time

def timer(func):
    """Mide el tiempo de ejecucion de una funcion."""
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        elapsed = time.time() - start
        print(f"[timer] '{func.__name__}' ejecutada en {elapsed:.4f}s")
        return result
    return wrapper

@timer
def process_trips(n):
    """Simula el procesamiento de n viajes."""
    import random
    return sum(random.randint(1, 50) for _ in range(n))

total = process_trips(100_000)
print(f"Total distancia simulada: {total}")

[timer] 'process_trips' ejecutada en 0.0504s
Total distancia simulada: 2549582


In [13]:
def validate_positive(func):
    """Valida que todos los argumentos sean numeros positivos."""
    def wrapper(*args, **kwargs):
        all_args = list(args) + list(kwargs.values())
        for v in all_args:
            if isinstance(v, (int, float)) and v <= 0:
                raise ValueError(f"Valor invalido: {v}. Deben ser positivos.")
        return func(*args, **kwargs)
    return wrapper

@validate_positive
def calculate_speed(distance_km, duration_min):
    """Calcula velocidad en km/h."""
    return distance_km / (duration_min / 60)

print(f"{calculate_speed(10, 30):.2f} km/h")  # OK

try:
    calculate_speed(-5, 30)  # Error
except ValueError as e:
    print(f"Error capturado: {e}")

20.00 km/h
Error capturado: Valor invalido: -5. Deben ser positivos.


### Decoradores apilados

Se puede aplicar más de un decorador. Se ejecutan de dentro hacia fuera.

In [14]:
@timer
@validate_positive
def heavy_calculation(distance_km, duration_min):
    """Calculo pesado con validacion y temporizador."""
    result = 0
    for _ in range(500_000):
        result += distance_km / (duration_min / 60)
    return result / 500_000

speed = heavy_calculation(8, 20)
print(f"Velocidad: {speed:.2f} km/h")

[timer] 'wrapper' ejecutada en 0.0345s


Velocidad: 24.00 km/h


---

## Resumen

| Concepto | Sintaxis | Uso típico |
|----------|----------|------------|
| `*args` | `def f(*args)` | Número variable de posicionales |
| `**kwargs` | `def f(**kwargs)` | Número variable de nominales |
| `map()` | `map(func, iterable)` | Transformar cada elemento |
| `filter()` | `filter(func, iterable)` | Filtrar por condición |
| Closure | `def outer(): def inner(): ...` | Encapsular estado |
| Decorador | `@decorator` | Añadir comportamiento reutilizable |

---

## Ejercicio

Crea un decorador `@retry(n)` que, si la función lanza una excepción,  
la reintente hasta `n` veces antes de propagar el error.

In [15]:
import random

def retry(n):
    """Decorador que reintenta una funcion hasta n veces."""
    def decorator(func):
        def wrapper(*args, **kwargs):
            for attempt in range(1, n + 1):
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    print(f"  Intento {attempt}/{n} fallido: {e}")
            raise RuntimeError(f"La funcion fallo tras {n} intentos.")
        return wrapper
    return decorator

@retry(n=3)
def fetch_station_data(station_id):
    """Simula una llamada a API que puede fallar."""
    if random.random() < 0.7:  # falla el 70% de las veces
        raise ConnectionError("Timeout al conectar con la API")
    return {"station": station_id, "bikes": random.randint(1, 20)}

random.seed(0)
try:
    data = fetch_station_data("SOL-01")
    print("Exito:", data)
except RuntimeError as e:
    print("Error final:", e)

Exito: {'station': 'SOL-01', 'bikes': 14}


---

**Anterior:** [02.01 - Funciones Básicas](./02_01_functions_basics.ipynb)  
**Siguiente:** [02.03 - Módulos y Paquetes](./02_03_modules_and_packages.ipynb)